# 0. SETUP

All notebooks need to run the cells up to `Helper Function` and then run only after a checkpoint at the start of a step.

In [ ]:
VERSION = 0
step = 0

## 0.0. Imports

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations

## 0.1. Path Definition

In [ ]:
!pwd
raw_data_path = '../data/raw'
interim_data_path = '../data/interim'
external_data_path = '../data/external'
figures_path = '../reports/figures'

## 0.2. Helper Functions

In [ ]:
def read_large_gzip_csv(path: str) -> pd.DataFrame:
    chunk = pd.read_csv(path,chunksize=1000000, compression='gzip')
    df = pd.concat(chunk);
    return df

def save_relevant_features():
    global relevant_features
    dirname = interim_data_path
    filename = 'relevant_features.csv'
    fullname =  dirname + '/' + filename

    rf = pd.Series(relevant_features)
    rf.to_csv(fullname)

def backup_df():
    global df
    global step

    dirname = interim_data_path + '/' + str(VERSION)
    filename = 'df' + str(step) + '.csv.gz'
    fullname =  dirname + '/' + filename

    output_dir = Path(dirname)
    output_dir.mkdir(parents=True, exist_ok=True)
    if not Path(fullname).exists():
      df.to_csv(fullname, compression='gzip', index=False)

def restore_df():
    global df
    global relevant_features
    global step

    dirname = interim_data_path
    filename = 'relevant_features.csv'
    fullname =  dirname + '/' + filename

    try: relevant_features
    except NameError:
        relevant_features = pd.read_csv(fullname, index_col=0)['0']

    dirname = interim_data_path + '/' + str(VERSION)
    filename = 'df' + str(step - 1) + '.csv.gz'
    fullname =  dirname + '/' + filename

    try: df
    except NameError:
        df = read_large_gzip_csv(fullname)
        for date in ['earliest_cr_line', 'sec_app_earliest_cr_line']:
           if date in df.columns:
              df[date] = pd.to_datetime(df[date], infer_datetime_format=True)

def show_data_types(df, relevant_features, type=None):
    if type:
        df = df.select_dtypes(type)

    pd.options.display.max_rows = None
    pd.options.display.max_colwidth  = 80
    variables_types = df.dtypes
    unique_values = pd.Series([df[i].unique() for i in df.columns], index=df.columns)

    data_types = pd.concat( [variables_types,
                                unique_values,
                                relevant_features], axis=1, join='inner',
                            keys=['types', 'unique values', 'description'] )
    data_types = data_types.rename_axis('names').sort_values(['types', 'names'])

    display(data_types)

def show_missing_data(df, relevant_features):
  pd.options.display.max_rows = None
  pd.options.display.max_colwidth  = 100
  num_rows = df.shape[0]
  total_missing = df.isna().sum()
  total_missing = total_missing[total_missing != 0].sort_values()

  missing_data = pd.concat( [total_missing,
                             (total_missing / num_rows) * 100,
                             relevant_features], axis=1, join='inner',
                           keys=['total missing', '%missing', 'description'] )

  display(missing_data)

def is_not_float(number):
  try:
    float(number)
    return False
  except:
    return True
  
def is_float(number):
  try:
    float(number)
    return True
  except:
    return False

def show_descriptive_statistical(df):
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(df.apply(np.mean)).T
    ct2 = pd.DataFrame(df.apply(np.median)).T

    # Dispersion - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(df.apply(np.std)).T
    d2 = pd.DataFrame(df.apply(min)).T
    d3 = pd.DataFrame(df.apply(max)).T
    d4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    display(m)

def ajust_parameters(column, min, max):
    init_size = df[column].shape[0]
    new_data = df[column][(df[column] >= min) & (df[column] <= max)]
    final_size = new_data.shape[0]
    print("Num rows")
    print("Initial: ", init_size)
    print("Final: ", final_size)
    print("Diff: ", init_size - final_size)
    print("Old skew: ", df[column].skew(), " - New skew: ", new_data.skew())
    print("Old kurtosis: ", df[column].kurtosis(), " - New kurtosis: ", new_data.kurtosis())
    plot_hist(new_data)

def show_unique_data(df):
  pd.options.display.max_rows = None
  pd.options.display.max_colwidth  = 100
  num_rows = df.shape[0]
  total_unique = df.nunique().sort_values()
  unique_data = pd.concat( [total_unique, (total_unique / num_rows) * 100],
                           axis=1,
                           keys=['total unique', 'total unique%'] )

  display(unique_data)

def plot_cat(df: pd.DataFrame, x: str, y: str, min=None, max=None, figpath=None):
  if min is None:
      min = df[y].min()
  if max is None:
      max = df[y].max()

  croped_df = df[(df[y] >= min) & (df[y] <= max)]
  for value in df[x].unique():
    print(x, "=", value, ": ", croped_df[y][croped_df[x] == value].mean())

  sns.catplot(
      data = croped_df,
      x=x, y=y, kind='boxen', scale='area'
  )

  if figpath is not None:
    plt.savefig(figpath,
              dpi=400)

def aesthetic_plot(plot, figpath=None, **kwargs):
  sns.set_style('white')
  sns.set_context('paper', font_scale=2.0)

  fig, ax = plt.subplots()

  plot(ax=ax, **kwargs)

  for axis in ['bottom', 'left']:
      ax.spines[axis].set_linewidth(2.5)
      ax.spines[axis].set_color('0.2')

  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)

  ax.grid(True, which='major', axis='both', alpha=1)

  ax.tick_params(axis='x', 
                bottom=True,
                which='major', 
                direction='out', 
                length=10, 
                width=5,
                color='0.2', 
                pad=15, 
                labelsize=15, 
                labelcolor='black', 
                labelrotation=15)

  ax.tick_params(axis='y', 
                left=True,
                which='major', 
                direction='out', 
                length=10, 
                width=5, 
                color='0.2', 
                pad=15, 
                labelsize=15, 
                labelcolor='black', 
                labelrotation=0)

  ax.set_xlabel(ax.get_xlabel(), fontsize=14, weight='bold')
  ax.set_ylabel(ax.get_ylabel(), fontsize=14, weight='bold')

  plt.xticks(size=14, weight='bold', color = '0.2')
  plt.yticks(size=14, weight='bold', color = '0.2')

  if figpath is not None:
    plt.savefig(figpath,
              dpi=400,
              facecolor=ax.get_facecolor())
   
def plot_hist(data: pd.Series, min=None, max=None, figpath=None):
  """
  Plot histogram of a Pandas' Series
  min, max: optionals, define the values boudries
  """

  if min is None:
      min = data.min()
  if max is None:
    max = data.max()

  aesthetic_plot(
      sns.histplot,
      figpath,
      data=data[(data >= min) & (data <= max)], 
              kde=True,
              bins=20,
              stat='density',
              multiple='layer',
              kde_kws={'bw_adjust':2},
              line_kws={'linewidth':3},
              palette='Set2'
  )

def plot_bar(data: pd.DataFrame, x: str, y: str, min=None, max=None, figpath=None):
  if min is None:
    min = data[y].min()
  if max is None:
    max = data[y].max()
  
  aesthetic_plot(
      sns.barplot,
      figpath,
      data=data[(data[y] >= min) & (data[y] <= max)],
      x=x,
      y=y,
      palette='Set2',
      capsize=0.5,
      edgecolor='0.2',
      lw=2.5
  )

def plot_bar(x: str, y: str, figpath=None):
  aesthetic_plot(
      sns.barplot,
      figpath,
      x=x,
      y=y,
      palette='Set2',
      capsize=0.5,
      edgecolor='0.2',
      lw=2.5
  )

def plot_count(data: pd.DataFrame, x: str, min=None, max=None, figpath=None):
  if min is None:
    min = data[x].min()
  if max is None:
    max = data[x].max()
  
  aesthetic_plot(
      sns.countplot,
      figpath,
      data=data[(data[x] >= min) & (data[x] <= max)],
      x=x,
      palette='Set2',
      capsize=0.5,
      edgecolor='0.2',
      lw=2.5
  )

def plot_box(data: pd.DataFrame, x: str, y: str, min=None, max=None, figpath=None):
  if min is None:
    min = data[y].min()
  if max is None:
    max = data[y].max()
  
  aesthetic_plot(
      sns.boxplot,
      figpath,
      data=data[(data[y] >= min) & (data[y] <= max)],
      x=x,
      y=y,
      palette='Set2'
)

def plot_scatter(data: pd.DataFrame, x: str, y: str, min=None, max=None, figpath=None):
  if min is None:
    min = data[y].min()
  if max is None:
    max = data[y].max()
  
  aesthetic_plot(
      sns.scatterplot,
      figpath,
      data=data[(data[y] >= min) & (data[y] <= max)],
      x=x,
      y=y,
      s=60,
      color='#000000',
      alpha=1
)

def variable_corr(dataset: pd.DataFrame, target: str):
  table = {}
  for column in dataset.columns:
      corr = pearsonr(dataset[column], dataset[target])[0]
      table[column] = corr
  df = pd.DataFrame(table, index=['correlation'])
  display(df.T.sort_values(by=['correlation']))

      

## 0.3. Loading Data

In [ ]:
df = read_large_gzip_csv(raw_data_path + '/accepted_2007_to_2018Q4.csv.gz')

# 1. DATA DESCRIPTION

In [ ]:
step = 1

## 1.0. Data Fields

This dataset has more than 150 features, but only some are relevant after the loan and, therefore, are unavailable during the moment of investment. The spreadsheet `LCDataDictionary.xlsx` available at Kaggle highlights the relevant features in the sheet `Browse Notes`. To facilitate the understanding of this notebook I'll add these features and it's descriptions in the dictionary variable bellow:

In [ ]:
relevant_features = {
  'acc_now_delinq': 'The number of accounts on which the borrower is now delinquent.',
  'acc_open_past_24mths': 'Number of trades opened in past 24 months.',
  'addr_state': 'The state provided by the borrower in the loan application',
  'all_util': 'Balance to credit limit on all trades',
  'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
  'annual_inc_joint': 'The combined self-reported annual income provided by the co-borrowers during registration',
  'application_type': 'Indicates whether the loan is an individual application or a joint application with two co-borrowers',
  'avg_cur_bal': 'Average current balance of all accounts',
  'bc_open_to_buy': 'Total open to buy on revolving bankcards.',
  'bc_util': 'Ratio of total current balance to high credit/credit limit for all bankcard accounts.',
  'chargeoff_within_12_mths': 'Number of charge-offs within 12 months',
  'collections_12_mths_ex_med': 'Number of collections in 12 months excluding medical collections',
  'delinq_2yrs': "The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years",
  'delinq_amnt': 'The past-due amount owed for the accounts on which the borrower is now delinquent.',
  'desc': 'Loan description provided by the borrower',
  'disbursement_method': 'The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY',
  'dti': 'A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.',
  'dti_joint': "A ratio calculated using the co-borrowers' total monthly payments on the total debt obligations, excluding mortgages and the requested LC loan, divided by the co-borrowers' combined self-reported monthly income",
  'earliest_cr_line': "The date the borrower's earliest reported credit line was opened",
  'emp_length': 'Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.',
  'emp_title': 'The job title supplied by the Borrower when applying for the loan.*',
  'fico_range_high': 'The upper boundary range the borrower’s FICO at loan origination belongs to.',
  'fico_range_low': 'The lower boundary range the borrower’s FICO at loan origination belongs to.',
  'funded_amnt': 'The total amount committed to that loan at that point in time.',
  'grade': 'LC assigned loan grade',
  'home_ownership': 'The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.',
  'id': 'A unique LC assigned ID for the loan listing.',
  'il_util': 'Ratio of total current balance to high credit/credit limit on all install acct',
  'initial_list_status': 'The initial listing status of the loan. Possible values are – W, F',
  'inq_fi': 'Number of personal finance inquiries',
  'inq_last_12m': 'Number of credit inquiries in past 12 months',
  'inq_last_6mths': 'The number of inquiries in past 6 months (excluding auto and mortgage inquiries)',
  'installment': 'The monthly payment owed by the borrower if the loan originates.',
  'int_rate': 'Interest Rate\xa0on the loan',
  'loan_amnt': 'The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.',
  'loan_status': 'Current status of the loan',
  'max_bal_bc': 'Maximum current balance owed on all revolving accounts',
  'member_id': 'A unique LC assigned Id for the borrower member.',
  'mo_sin_old_il_acct': 'Months since oldest bank installment account opened',
  'mo_sin_old_rev_tl_op': 'Months since oldest revolving account opened',
  'mo_sin_rcnt_rev_tl_op': 'Months since most recent revolving account opened',
  'mo_sin_rcnt_tl': 'Months since most recent account opened',
  'mort_acc': 'Number of mortgage accounts.',
  'mths_since_last_delinq': "The number of months since the borrower's last delinquency.",
  'mths_since_last_major_derog': 'Months since most recent 90-day or worse rating',
  'mths_since_last_record': 'The number of months since the last public record.',
  'mths_since_rcnt_il': 'Months since most recent installment accounts opened',
  'mths_since_recent_bc': 'Months since most recent bankcard account opened.',
  'mths_since_recent_bc_dlq': 'Months since most recent bankcard delinquency',
  'mths_since_recent_inq': 'Months since most recent inquiry.',
  'mths_since_recent_revol_delinq': 'Months since most recent revolving delinquency.',
  'num_accts_ever_120_pd': 'Number of accounts ever 120 or more days past due',
  'num_actv_bc_tl': 'Number of currently active bankcard accounts',
  'num_actv_rev_tl': 'Number of currently active revolving trades',
  'num_bc_sats': 'Number of satisfactory bankcard accounts',
  'num_bc_tl': 'Number of bankcard accounts',
  'num_il_tl': 'Number of installment accounts',
  'num_op_rev_tl': 'Number of open revolving accounts',
  'num_rev_accts': 'Number of revolving accounts',
  'num_rev_tl_bal_gt_0': 'Number of revolving trades with balance >0',
  'num_sats': 'Number of satisfactory accounts',
  'num_tl_120dpd_2m': 'Number of accounts currently 120 days past due (updated in past 2 months)',
  'num_tl_30dpd': 'Number of accounts currently 30 days past due (updated in past 2 months)',
  'num_tl_90g_dpd_24m': 'Number of accounts 90 or more days past due in last 24 months',
  'num_tl_op_past_12m': 'Number of accounts opened in past 12 months',
  'open_acc': "The number of open credit lines in the borrower's credit file.",
  'open_acc_6m': 'Number of open trades in last 6 months',
  'open_act_il': 'Number of currently active installment trades',
  'open_il_12m': 'Number of installment accounts opened in past 12 months',
  'open_il_24m': 'Number of installment accounts opened in past 24 months',
  'open_rv_12m': 'Number of revolving trades opened in past 12 months',
  'open_rv_24m': 'Number of revolving trades opened in past 24 months',
  'pct_tl_nvr_dlq': 'Percent of trades never delinquent',
  'percent_bc_gt_75': 'Percentage of all bankcard accounts > 75% of limit.',
  'pub_rec': 'Number of derogatory public records',
  'pub_rec_bankruptcies': 'Number of public record bankruptcies',
  'purpose': 'A category provided by the borrower for the loan request. ',
  'revol_bal': 'Total credit revolving balance',
  'revol_bal_joint': ' Sum of revolving credit balance of the co-borrowers, net of duplicate balances',
  'revol_util': 'Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.',
  'sec_app_chargeoff_within_12_mths': ' Number of charge-offs within last 12 months at time of application for the secondary applicant',
  'sec_app_collections_12_mths_ex_med': ' Number of collections within last 12 months excluding medical collections at time of application for the secondary applicant',
  'sec_app_earliest_cr_line': ' Earliest credit line at time of application for the secondary applicant',
  'sec_app_fico_range_high': ' FICO range (low) for the secondary applicant',
  'sec_app_fico_range_low': ' FICO range (high) for the secondary applicant',
  'sec_app_inq_last_6mths': ' Credit inquiries in the last 6 months at time of application for the secondary applicant',
  'sec_app_mort_acc': ' Number of mortgage accounts at time of application for the secondary applicant',
  'sec_app_mths_since_last_major_derog': ' Months since most recent 90-day or worse rating at time of application for the secondary applicant',
  'sec_app_num_rev_accts': ' Number of revolving accounts at time of application for the secondary applicant',
  'sec_app_open_acc': ' Number of open trades at time of application for the secondary applicant',
  'sec_app_open_act_il': ' Number of currently active installment trades at time of application for the secondary applicant',
  'sec_app_revol_util': ' Ratio of total current balance to high credit/credit limit for all revolving accounts',
  'sub_grade': 'LC assigned loan subgrade',
  'tax_liens': 'Number of tax liens',
  'term': 'The number of payments on the loan. Values are in months and can be either 36 or 60.',
  'title': 'The loan title provided by the borrower',
  'tot_coll_amt': 'Total collection amounts ever owed',
  'tot_cur_bal': 'Total current balance of all accounts',
  'tot_hi_cred_lim': 'Total high credit/credit limit',
  'total_acc': "The total number of credit lines currently in the borrower's credit file",
  'total_bal_ex_mort': 'Total credit balance excluding mortgage',
  'total_bal_il': 'Total current balance of all installment accounts',
  'total_bc_limit': 'Total bankcard high credit/credit limit',
  'total_cu_tl': 'Number of finance trades',
  'total_il_high_credit_limit': 'Total installment high credit/credit limit',
  'total_rev_hi_lim': 'Total revolving high credit/credit limit',
  'url': 'URL for the LC page with listing data.',
  'verification_status': 'Indicates if income was verified by LC, not verified, or if the income source was verified',
  'verification_status_joint': "Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified",
  'zip_code': 'The first 3 numbers of the zip code provided by the borrower in the loan application.'}

relevant_features = pd.Series(relevant_features)
save_relevant_features()

In [ ]:
df = df[relevant_features.keys()].copy()

## 1.1. Data Dimensions

In [ ]:
print(f'Number of lines: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')

## 1.2. Data Types

### 1.2.0 Type List

In [ ]:
show_data_types(df, relevant_features)

### 1.2.1 Type Changes

`earliest_cr_line` and `sec_app_earliest_cr_line` can be changed into data format.

In [ ]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format=True)
df['sec_app_earliest_cr_line'] = pd.to_datetime(df['sec_app_earliest_cr_line'], infer_datetime_format=True)

`emp_length`,  `id` and `term` can be transformed into numerical values. In case of `emp_length` the value of `10+ years` and `< 1 year` will be transformed in `11` and `0` respectively.

Some `id` values contain strings and isn't possible to transform them into float. Analysing these rows it appears that these values are null and these rows can be excluded.

In [ ]:
id_not_float = df.loc[df['id'].apply(is_not_float)]
id_not_float.head()

In [ ]:
df = df.loc[df['id'].apply(is_float)]

In [ ]:
df['emp_length'] = df['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
df['emp_length'] = df['emp_length'].str.extract('(\d+)').astype('float')
df['id'] = df['id'].astype('float')
df['term'] = df['term'].str.extract('(\d+)').astype('float')

In [ ]:
df[['emp_length', 'id', 'term']].head()

## 1.3. NA Variables

### 1.3.0. Check NA

In [ ]:
show_missing_data(df, relevant_features)

### 1.3.1. Fillout NA

I'll start by eliminating every column with more than 40% of null variables:

In [ ]:
df = df.loc[:, df.isna().mean() < .4]
df.shape

I'll drop every row with null values

In [ ]:
df = df.dropna()
df.shape

## 1.4. Descriptive Statistics

In [ ]:
num_attributes = df.select_dtypes(include=['int64', 'float64'])
cat_attributes = df.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.4.0. Numerical Attributes

In [ ]:
show_descriptive_statistical(num_attributes)

I select the variables with high absolute value of skew and kurtosis to take a look. The threshold for skew is 10 and kurtosis is 50:

In [ ]:
bad_skew_kurtosis = num_attributes.loc[:, (num_attributes.apply(lambda x: x.skew()) > 10) | (num_attributes.apply(lambda x: x.kurtosis()) > 50)].columns
bad_skew_kurtosis

#### acc_now_delinq

In [ ]:
plot_hist(df.acc_now_delinq)

In [ ]:
ajust_parameters('acc_now_delinq', 1, 50)

I'll leave this as is.

#### annual_inc

In [ ]:
plot_hist(df.annual_inc)

Ajusting parameters:

In [ ]:
ajust_parameters('annual_inc', 0, 5e5)

In [ ]:
(1974/1027854 )*100

In [ ]:
df = df[df['annual_inc'] <= 5e5]

#### chargeoff_within_12_mths

In [ ]:
plot_hist(df.chargeoff_within_12_mths)

In [ ]:
ajust_parameters('chargeoff_within_12_mths', 1, 100)

I'll leave this as is.

#### collections_12_mths_ex_med

In [ ]:
plot_hist(df.collections_12_mths_ex_med)

In [ ]:
ajust_parameters('collections_12_mths_ex_med', 1, 100)

I'll leave this as is.

#### delinq_2yrs

In [ ]:
plot_hist(df.delinq_2yrs)

In [ ]:
ajust_parameters('delinq_2yrs', 1, 30)

I'll leave this as is.

#### delinq_amnt

In [ ]:
plot_hist(df.delinq_amnt)

In [ ]:
ajust_parameters('delinq_amnt', 1, 1000000)

I'll leave this as is.

#### dti

In [ ]:
plot_hist(df.dti)

In [ ]:
ajust_parameters('dti', 0, 100)

Any value less than 0 doesn't make sense.
It's strange to have very high values. Will crop these values now and do a better investigation later.

In [ ]:
df = df[(df['dti'] >= 0) & (df['dti'] <= 100)]

#### max_bal_bc

In [ ]:
plot_hist(df.max_bal_bc)

In [ ]:
ajust_parameters('max_bal_bc', 0, 1e5)

In [ ]:
df = df[df['max_bal_bc'] <= 1e5]

#### num_accts_ever_120_pd

In [ ]:
plot_hist(df.num_accts_ever_120_pd)

In [ ]:
ajust_parameters('num_accts_ever_120_pd', 1, 100)

I'll it leave as is.

#### num_tl_120dpd_2m

In [ ]:
plot_hist(df.num_tl_120dpd_2m)

I'll leave it as is.

#### num_tl_30dpd

In [ ]:
plot_hist(df.num_tl_30dpd)

I'll leave it as is.

#### num_tl_90g_dpd_24m

In [ ]:
plot_hist(df.num_tl_90g_dpd_24m)

I'll leave it as is.

#### pub_rec

In [ ]:
plot_hist(df.pub_rec)

I'll leave it as is.

#### revol_bal

In [ ]:
plot_hist(df.revol_bal)

I'll leave it as is.

In [ ]:
ajust_parameters('revol_bal', 0, 2e5)

In [ ]:
df = df[df['revol_bal'] <= 2e5]

#### tax_liens

In [ ]:
plot_hist(df.tax_liens)

I'll leave it as is.

#### tot_coll_amt

In [ ]:
plot_hist(df.tot_coll_amt)

I'll leave it as is.

#### tot_hi_cred_lim

In [ ]:
plot_hist(df.tot_hi_cred_lim)

In [ ]:
ajust_parameters('tot_hi_cred_lim', 0, 4e6)

I'll leave it as is.

#### total_rev_hi_lim

In [ ]:
plot_hist(df.total_rev_hi_lim)

In [ ]:
ajust_parameters('total_rev_hi_lim', 0, 5e5)

I'll leave it as is.

### 1.4.1. Categorical Attributes

In [ ]:
show_unique_data(cat_attributes)

`emp_title` and `url` have a high cardinality for categorical atributes, therefore they are going to be eliminated.

In [ ]:
df = df.drop(['emp_title', 'url'], axis=1)

In [ ]:
backup_df()

# 2. FEATURE ENGINEERING

In [ ]:
step = 2
restore_df()

## 2.0. Hypothesis Mind Map

In [ ]:
show_data_types(df, relevant_features)

### 2.0.0. Target feature

A target feature deste dataset é `loan_status`. The description for each status is provided by the Lending Club:

*   Current: Loan is up to date on all outstanding payments.
*   In Grace Period: Loan is past due but within the 15-day grace period.
*   Late (16-30): Loan has not been current for 16 to 30 days.
*   Late (31-120): Loan has not been current for 31 to 120 days.
*   Fully paid: Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.
*   Default: Loan has not been current for an extended period of time.
*   Charged Off: Loan for which there is no longer a reasonable expectation of further payments.

### 2.0.1. Mind Map

In [ ]:
Image(external_data_path + '/Loan_status.png')

## 2.1. Creating hypotheses

### 2.1.0. Income and assets hypotheses

**1.** Clients with higher income should have good loan status.

**2.** Clients with higher account balance should have good good loan status.

### 2.1.1. Debt and unpaid bills hypotheses

**3.** Clients with higher number of unpaid debts should have bad loan status.

**4.** Clients with higher number of collections should have bad loan status.

**5.** Clients with higher number of inquiries should have bad loan status.

### 2.1.2. Accounts status hypotheses

**6.** Clients with higher number of delinquencies should have bad loan status.

**7.** Clients with higher credit should have good loan status.

**8.** Clients using more credit should have good loan status.

### 2.1.3. Client hypotheses

**9.** Clients with higher amounts of trades should have good loan status.

**10.** Clients with higher employment lenght should have good loan status.

**11.** Clients that are older clients should have good loan status.

**12.** Clients that are newer clients should have bad loan status.

**13.** Clients with more accounts should have good loan status.

**14.** Clients with bad public record should have bad loan status.

### 2.1.4. Other hypotheses

**15.** Clients with high FICO score should have good loan status.

**16.** Clients with high loan amount should have bad loan status.

**17.** Clients with higher interest rates on the loan should have bad loan status.

**18.** Clients with higher number of payments on the loan should have bad loan status.

## 2.2. Feature Engineering

`loan_status` is a categorical variable with the following values:

In [ ]:
df['loan_status'].value_counts()

These values will be separated into good values (1) and bad values (0). We'll consider `Current`, `Fully Paid` and `In Grace Period` as good and the rest as bad and put these values in the variable `loan_status_gb`:

In [ ]:
df['loan_status_gb'] = df['loan_status'].isin(['Current', 'Fully Paid', 'In Grace Period']).astype('int')
df['loan_status_gb'].value_counts()

This dataset is highly imbalanced.

In [ ]:
backup_df()

# 3. VARIABLE FILTERING

In [ ]:
step = 3
restore_df()

No filtering in this version.

In [ ]:
backup_df()

# 4. EXPLORATORY DATA ANALISIS

In [ ]:
step = 4
restore_df()

In [ ]:
num_attributes = df.select_dtypes(include=['int64', 'float64'])
cat_attributes = df.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

## 4.0. Numerical variables

## 4.0. Univariate Analisis

## 4.1. Bivariate Analisis

### 4.1.0. Numerical attributes with numerical attributes

Printing variables with more than 90% correlation:

In [ ]:
high_correlation_variables = []
variables = num_attributes.columns.tolist()
for i in range(len(variables)):
    for j in range(i + 1, len(variables)):
        correlation = pearsonr(num_attributes[variables[i]], num_attributes[variables[j]])[0]
        if correlation >= 0.9:
            print(variables[i], '-', variables[j], ': ', correlation)
            if variables[i] not in high_correlation_variables: high_correlation_variables.append(variables[i])
            if variables[j] not in high_correlation_variables: high_correlation_variables.append(variables[j])

In [ ]:
df[high_correlation_variables].head(10)

Identical or very similar variables are going to be eliminated:

In [ ]:
df = df.drop(['fico_range_high', 'funded_amnt', 'installment', 'num_actv_rev_tl', 'num_sats', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_il_high_credit_limit'], axis=1)
num_attributes = df.select_dtypes(include=['int64', 'float64'])

In [ ]:
variable_corr(num_attributes, 'loan_status_gb')

Only variables with a correlation higher than 4% with the target feature will remain.

In [ ]:
low_corr = []
for column in num_attributes.columns:
    corr = pearsonr(num_attributes[column], num_attributes['loan_status_gb'])[0]
    if abs(corr) < 0.04:
        low_corr.append(column)

print(low_corr)
df = df.drop(low_corr, axis=1)
num_attributes = df.select_dtypes(include=['int64', 'float64'])

In [ ]:
show_data_types(num_attributes, relevant_features)

In [ ]:
output_dir = Path(figures_path)
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
figpath = figures_path + '/acc_open_past_24mths' + '-box'
plot_box(df, x='loan_status_gb', y='acc_open_past_24mths', figpath=figpath)

In [ ]:
figpath = figures_path + '/acc_open_past_24mths' + '-cat'
plot_cat(df, x='loan_status_gb', y='acc_open_past_24mths', figpath=figpath)

In [ ]:
figpath = figures_path + '/all_util' + '-box'
plot_box(df, x='loan_status_gb', y='all_util', figpath=figpath)

In [ ]:
figpath = figures_path + '/all_util' + '-cat'
plot_cat(df, x='loan_status_gb', y='all_util', figpath=figpath)

In [ ]:
figpath = figures_path + '/annual_inc' + '-box'
plot_box(df, x='loan_status_gb', y='annual_inc', figpath=figpath)

In [ ]:
figpath = figures_path + '/annual_inc' + '-cat'
plot_cat(df, x='loan_status_gb', y='annual_inc', figpath=figpath)

In [ ]:
figpath = figures_path + '/avg_cur_bal' + '-box'
plot_box(df, x='loan_status_gb', y='avg_cur_bal', figpath=figpath)

In [ ]:
figpath = figures_path + '/avg_cur_bal' + '-cat'
plot_cat(df, x='loan_status_gb', y='avg_cur_bal', figpath=figpath)

In [ ]:
figpath = figures_path + '/bc_open_to_buy' + '-box'
plot_box(df, x='loan_status_gb', y='bc_open_to_buy', figpath=figpath)

In [ ]:
figpath = figures_path + '/bc_open_to_buy' + '-cat'
plot_cat(df, x='loan_status_gb', y='bc_open_to_buy', figpath=figpath)

In [ ]:
figpath = figures_path + '/bc_util' + '-box'
plot_box(df, x='loan_status_gb', y='bc_util', figpath=figpath)

In [ ]:
figpath = figures_path + '/bc_util' + '-cat'
plot_cat(df, x='loan_status_gb', y='bc_util', figpath=figpath)

In [ ]:
figpath = figures_path + '/dti' + '-box'
plot_box(df, x='loan_status_gb', y='dti', figpath=figpath)

In [ ]:
figpath = figures_path + '/dti' + '-cat'
plot_cat(df, x='loan_status_gb', y='dti', figpath=figpath)

In [ ]:
figpath = figures_path + '/fico_range_low' + '-box'
plot_box(df, x='loan_status_gb', y='fico_range_low', figpath=figpath)

In [ ]:
figpath = figures_path + '/fico_range_low' + '-cat'
plot_cat(df, x='loan_status_gb', y='fico_range_low', figpath=figpath)

In [ ]:
figpath = figures_path + '/inq_fi' + '-box'
plot_box(df, x='loan_status_gb', y='inq_fi', figpath=figpath)

In [ ]:
figpath = figures_path + '/inq_fi' + '-cat'
plot_cat(df, x='loan_status_gb', y='inq_fi', figpath=figpath)

In [ ]:
figpath = figures_path + '/inq_last_12m' + '-box'
plot_box(df, x='loan_status_gb', y='inq_last_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/inq_last_12m' + '-cat'
plot_cat(df, x='loan_status_gb', y='inq_last_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/inq_last_6mths' + '-box'
plot_box(df, x='loan_status_gb', y='inq_last_6mths', figpath=figpath)

In [ ]:
figpath = figures_path + '/inq_last_6mths' + '-cat'
plot_cat(df, x='loan_status_gb', y='inq_last_6mths', figpath=figpath)

In [ ]:
figpath = figures_path + '/int_rate' + '-box'
plot_box(df, x='loan_status_gb', y='int_rate', figpath=figpath)

In [ ]:
figpath = figures_path + '/int_rate' + '-cat'
plot_cat(df, x='loan_status_gb', y='int_rate', figpath=figpath)

In [ ]:
figpath = figures_path + '/max_bal_bc' + '-box'
plot_box(df, x='loan_status_gb', y='max_bal_bc', figpath=figpath)

In [ ]:
figpath = figures_path + '/max_bal_bc' + '-cat'
plot_cat(df, x='loan_status_gb', y='max_bal_bc', figpath=figpath)

In [ ]:
figpath = figures_path + '/mo_sin_old_rev_tl_op' + '-box'
plot_box(df, x='loan_status_gb', y='mo_sin_old_rev_tl_op', figpath=figpath)

In [ ]:
figpath = figures_path + '/mo_sin_old_rev_tl_op' + '-cat'
plot_cat(df, x='loan_status_gb', y='mo_sin_old_rev_tl_op', figpath=figpath)

In [ ]:
figpath = figures_path + '/mo_sin_rcnt_rev_tl_op' + '-box'
plot_box(df, x='loan_status_gb', y='mo_sin_rcnt_rev_tl_op', figpath=figpath)

In [ ]:
figpath = figures_path + '/mo_sin_rcnt_rev_tl_op' + '-cat'
plot_cat(df, x='loan_status_gb', y='mo_sin_rcnt_rev_tl_op', figpath=figpath)

In [ ]:
figpath = figures_path + '/mo_sin_rcnt_tl' + '-box'
plot_box(df, x='loan_status_gb', y='mo_sin_rcnt_tl', figpath=figpath)

In [ ]:
figpath = figures_path + '/mo_sin_rcnt_tl' + '-cat'
plot_cat(df, x='loan_status_gb', y='mo_sin_rcnt_tl', figpath=figpath)

In [ ]:
figpath = figures_path + '/mort_acc' + '-box'
plot_box(df, x='loan_status_gb', y='mort_acc', figpath=figpath)

In [ ]:
figpath = figures_path + '/mort_acc' + '-cat'
plot_cat(df, x='loan_status_gb', y='mort_acc', figpath=figpath)

In [ ]:
figpath = figures_path + '/mths_since_recent_bc' + '-box'
plot_box(df, x='loan_status_gb', y='mths_since_recent_bc', figpath=figpath)

In [ ]:
figpath = figures_path + '/mths_since_recent_bc' + '-cat'
plot_cat(df, x='loan_status_gb', y='mths_since_recent_bc', figpath=figpath)

In [ ]:
figpath = figures_path + '/mths_since_recent_inq' + '-box'
plot_box(df, x='loan_status_gb', y='mths_since_recent_inq', figpath=figpath)

In [ ]:
figpath = figures_path + '/mths_since_recent_inq' + '-cat'
plot_cat(df, x='loan_status_gb', y='mths_since_recent_inq', figpath=figpath)

In [ ]:
figpath = figures_path + '/num_tl_op_past_12m' + '-box'
plot_box(df, x='loan_status_gb', y='num_tl_op_past_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/num_tl_op_past_12m' + '-cat'
plot_cat(df, x='loan_status_gb', y='num_tl_op_past_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_acc_6m' + '-box'
plot_box(df, x='loan_status_gb', y='open_acc_6m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_acc_6m' + '-cat'
plot_cat(df, x='loan_status_gb', y='open_acc_6m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_il_12m' + '-box'
plot_box(df, x='loan_status_gb', y='open_il_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_il_12m' + '-cat'
plot_cat(df, x='loan_status_gb', y='open_il_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_il_24m' + '-box'
plot_box(df, x='loan_status_gb', y='open_il_24m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_il_24m' + '-cat'
plot_cat(df, x='loan_status_gb', y='open_il_24m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_rv_12m' + '-box'
plot_box(df, x='loan_status_gb', y='open_rv_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_rv_12m' + '-cat'
plot_cat(df, x='loan_status_gb', y='open_rv_12m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_rv_24m' + '-box'
plot_box(df, x='loan_status_gb', y='open_rv_24m', figpath=figpath)

In [ ]:
figpath = figures_path + '/open_rv_24m' + '-cat'
plot_cat(df, x='loan_status_gb', y='open_rv_24m', figpath=figpath)

In [ ]:
figpath = figures_path + '/percent_bc_gt_75' + '-box'
plot_box(df, x='loan_status_gb', y='percent_bc_gt_75', figpath=figpath)

In [ ]:
figpath = figures_path + '/percent_bc_gt_75' + '-cat'
plot_cat(df, x='loan_status_gb', y='percent_bc_gt_75', figpath=figpath)

In [ ]:
figpath = figures_path + '/revol_util' + '-box'
plot_box(df, x='loan_status_gb', y='revol_util', figpath=figpath)

In [ ]:
figpath = figures_path + '/revol_util' + '-cat'
plot_cat(df, x='loan_status_gb', y='revol_util', figpath=figpath)

In [ ]:
figpath = figures_path + '/term' + '-box'
plot_box(df, x='loan_status_gb', y='term', figpath=figpath)

In [ ]:
figpath = figures_path + '/term' + '-cat'
plot_cat(df, x='loan_status_gb', y='term', figpath=figpath)

In [ ]:
figpath = figures_path + '/tot_cur_bal' + '-box'
plot_box(df, x='loan_status_gb', y='tot_cur_bal', figpath=figpath)

In [ ]:
figpath = figures_path + '/tot_cur_bal' + '-cat'
plot_cat(df, x='loan_status_gb', y='tot_cur_bal', figpath=figpath)

In [ ]:
figpath = figures_path + '/total_bc_limit' + '-box'
plot_box(df, x='loan_status_gb', y='total_bc_limit', figpath=figpath)

In [ ]:
figpath = figures_path + '/total_bc_limit' + '-cat'
plot_cat(df, x='loan_status_gb', y='total_bc_limit', figpath=figpath)

In [ ]:
figpath = figures_path + '/total_rev_hi_lim' + '-box'
plot_box(df, x='loan_status_gb', y='total_rev_hi_lim', figpath=figpath)

In [ ]:
figpath = figures_path + '/total_rev_hi_lim' + '-cat'
plot_cat(df, x='loan_status_gb', y='total_rev_hi_lim', figpath=figpath)

### 4.1.1. Numerical attributes with categorical attributes

In [ ]:
plot_hist(df['int_rate'])

In [ ]:
grade_count = df['grade'].value_counts().reindex(['A','B','C','D','E','F','G'])
plot_bar(x=grade_count.index, y=grade_count)
#sns.barplot(x=grade_count.index, y=grade_count)

In [ ]:
sns.histplot(data=df, x='int_rate', hue='grade', log_scale=(None, True))

Through these grafics it's possible to see that `grade` and `int_rate` are correlated. So we'll be removing `int_rate`:

In [ ]:
df.drop('int_rate', axis=1, inplace=True)
num_attributes = df.select_dtypes(include=['int64', 'float64'])

### 4.1.2. Categorical attributes with categorical attributes

In [ ]:
show_data_types(cat_attributes, relevant_features)

In [ ]:
comb_cat_feat = np.array(list(combinations(cat_attributes, 2)))
corr_cat_feat = np.array([])
for comb in comb_cat_feat:
    table = pd.pivot_table(cat_attributes, index=comb[0], columns=comb[1], aggfunc='count').fillna(0)
    corr = np.sqrt(chi2_contingency(table)[0] / (table.values.sum() * (np.min(table.shape) - 1) ) )
    corr_cat_feat = np.append(corr_cat_feat, corr)
high_corr_cat = comb_cat_feat[corr_cat_feat >= 0.9]
high_corr_cat

In [ ]:
cat_attributes[['addr_state', 'zip_code', 'grade', 'sub_grade', 'purpose', 'title']].head(10)

In [ ]:
df.drop(['zip_code', 'sub_grade', 'title'], axis=1, inplace=True)

## Multivatiate Analisis

In [ ]:
backup_df()